# Importar todas las librerias necesarias

In [ ]:
from pathlib import Path
import pandas as pd
import duckdb

directorio = Path(
    "//EURFL01/advisors.hal/non-hudson/Coral Homes/CoralHudson/6. Stock/Parquet"
).absolute()

# Crear base de datos local, sobre la cual se escribira y actualizaran datos

In [ ]:
bd = duckdb.connect("basedatos_wholesale.db")
bd.execute(
    f"CREATE TABLE master_tape AS SELECT * FROM parquet_scan('{directorio}/master_tape.parquet')"
)
bd.execute(
    f"CREATE TABLE offers_data AS SELECT * FROM parquet_scan('{directorio}/offers.parquet')"
)
bd.execute(
    f"CREATE TABLE ws_segregated AS SELECT * FROM parquet_scan('{directorio}/disaggregated_assets.parquet')"
)

# 🦆 DUCKDB: Pruebas

## Guardar nueva base de datos

In [ ]:
import duckdb
from pathlib import Path

DIR_PARQUET = Path("N:/CoralHudson/6. Stock/Parquet").absolute()
DATABASE_FILE = Path("N:/CoralHudson/1. AM/8. Wholesale Channel/.code/database.db")
parquet_files = Path(DIR_PARQUET).glob("*.parquet")

with duckdb.connect(DATABASE_FILE.as_posix()) as db:
    df = db.execute("show tables").df()

df

## Seleccionar las tablas que coincidan en patron de nombre y traerlas de vuelta como lista

In [ ]:
DATABASE_FILE = Path("C:/Users/amontes/Projects/wholesale/basedatos_wholesale.db")

with duckdb.connect(DATABASE_FILE.as_posix()) as db:
    data = db.execute(
        "select table_name from information_schema.tables where regexp_matches(table_name, 'ws_.+_offers')"
    ).fetchall()
    db.close()

data = [d[0] for d in data]
data

## Crear tabla temporal usando fichero query y hacer joins en memoria con otras

In [ ]:
import duckdb

with open("./queries/unnest_unique_urs.sql", encoding="utf8") as sql_file:
    query = sql_file.read()
    improved_query = f"create temp table my_temp_attempt as {query}"

with duckdb.connect("./basedatos_wholesale.db") as db:
    db.execute(improved_query)
    df = db.execute("select coalesce(columns('all')) from my_temp_attempt").df()

df

## Performance glob vs os: Listar ficheros en directorio

In [ ]:
import time
import glob
import os


def timing(f):
    def wrap(*args):
        start_time = time.perf_counter()
        ret = f(*args)
        end_time = time.perf_counter() - start_time
        print(f"{f.__name__} function took {end_time:0.3f} seconds")
        return ret

    return wrap


def scan_os(dir_path: str):
    files_list = []
    for dir in os.scandir(dir_path):
        for item, subdirs, files in os.walk(dir):
            files_list += files
    return files_list


def scan_glob(dir_path: str):
    files = glob.iglob(dir_path + "/**/[!~$]*.xlsx", recursive=True)
    files_list = [Path(f).absolute() for f in files]
    return files_list


str_path = "N:/Coralhudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2023/"

# %timeit scan_os(str_path)
# %timeit scan_glob(str_path)
# df.full_path.tolist()

## Update de ofertas incremental: ofertas ya en fichero

In [ ]:
import duckdb
from pathlib import Path

with duckdb.connect("./basedatos_wholesale.db") as db:
    listado_db = db.execute(
        "select distinct full_path from ws_current_offers"
    ).fetchall()

listado_db = [Path(l[0]).as_posix() for l in listado_db]
listado_dir = scan_glob(str_path)

In [ ]:
from rich import print

print(listado_db[4])
print(listado_dir[4])
print(listado_db[4] == listado_dir[4])

In [ ]:
list = [12, 12, 32]
list2 = [2809, 19010, 10912]
list += list2
print(list)

## LSEV/PPA aggregate data: memoria, eliminar las columnas duplicadas ("_2"?) y traer sólo las nuevas

In [ ]:
import duckdb

DATABASE_FILE = Path("C:/Users/amontes/Projects/wholesale/basedatos_wholesale.db")

query = """with all_offers as (
  select * from ws_hist_offers w1
  union all
  select * from ws_current_offers w2
),
-- que queremos traer del tape de ofertas semanal??
-- tenemos que traerlo por offer_id?
-- NOTE:
-- no, hay que traer los agregados por offer_id pero no lincar via offer_id, me explico?
-- los conteos de urs_per_offers_table y urs_per_file son IGUALES de modo que todo está ok
-- WARN: Error en ws_hist_offers porque no existe, hay que meter un algo para que solo pille las disponibles,
-- tal vez un regex que pille las ws_
enriched_offers as (
  select  a.*,
          array_agg(DISTINCT o.ur_current) AS all_urs_pot,
          array_agg(DISTINCT o.commercialdev) AS all_commercialdevs_pot,
          array_agg(DISTINCT o.jointdev) AS all_jointdevs_pot,
          array_agg(DISTINCT o.offerstatus) AS offer_status
  from all_offers a
  left join offers_data o
  on a.offer_id = o.offerid
  group by all
),
final_piece as(select
    w.*,
    l1.unique_urs,
    l1.commercialdevs,
    l1.jointdevs,
    count(l1.unique_urs) as total_urs,
from enriched_offers as w,
    lateral (
        select list_distinct(list_concat(w.unique_urs,w.all_urs_pot)),
               cast(list_distinct(list_concat(w.commercialdev,w.all_commercialdevs_pot)) as int[]),
               cast(list_distinct(list_concat(w.jointdev,w.all_jointdevs_pot)) as int[])
    ) l1(unique_urs, commercialdevs, jointdevs)
group by all)
select  f.unique_id,
        cast(f.offer_id as int) as offer_id,
        f.offer_status[1] as offer_status,
        list_aggregate(f.unique_urs, 'string_agg', ' | ') as unique_urs,
        list_aggregate(f.commercialdevs, 'string_agg', ' | ') as commercialdevs,
        list_aggregate(f.jointdevs, 'string_agg', ' | ') as jointdevs,
        f.total_urs,
        sum(m.lsev_dec19) as lsevdec19,
        sum(m.ppa) as ppa,
        mode(m.bucketi_ha) as bucket,
from final_piece f
cross join unnest(f.unique_urs) as r (urs)
left join master_tape as m
    on r.urs = m.ur_current
group by all
"""

with duckdb.connect(DATABASE_FILE.as_posix()) as db:
    db.sql(query).show()

## Reusar tape ofertas anterior
Extraer info y probar a 1) concatenar o 2) combinar, pese a que falten offerids

In [ ]:
import duckdb
import pandas as pd

fichero = "//EURFL01/advisors.hal/non-hudson/Coral Homes/CoralHudson/1. AM/8. Wholesale Channel/.outputs/#20230720_Coral_Homes_Offers_Data.xlsx"

tape_anterior = pd.read_excel(fichero, skiprows=5).convert_dtypes()

with duckdb.connect(DATABASE_FILE.as_posix()) as db:
    db.register("tmp_prev_tape", tape_anterior)
    db.execute(
        "create temp table tmp_current_tape as select * from ws_current_offers union all select * from ws_hist_offers"
    )
    df = db.execute(
        """with all_data as (select *,
                    from tmp_current_tape t1 
                    left join tmp_prev_tape t2 
                    on t1.unique_id = t2.unique_id)
                    select columns(x -> x not similar to '.+:1') from all_data;"""
    ).df()

df.to_clipboard()

In [8]:
import re
import datetime

# Obtener el {current_year} para escanear las ofertas
cy = datetime.datetime.now().year
list_years = ["2023", "2022", "2021", "2012"]
pattern = re.compile(rf"^({cy})$")
print(list(filter(pattern.search, list_years)))
print([x for x in list_years if not pattern.match(x)])


# Probemos el regex en condiciones
listado_columnas = ["asdf", "asdf:1", "otra", "otra:1"]

patron = re.compile(":1")
list(filter(patron.search, listado_columnas))
print(str(cy)[-1])

['2023']
['2022', '2021', '2012']
3


In [1]:
def test_func(trigger: bool) -> list | bool:
    if trigger:
        return [1, 2, 3]
    else:
        return False


var1 = test_func(False)
if var1:
    print("Los false cuentan como true, en comprobación simple!")
else:
    print("Los false cuentan como false, en comprobación simple!")

Los false cuentan como false, en comprobación simple!


### Funciona, asi que ahora probaremos con encontrar el ultimo fichero

In [ ]:
import re
import os
import datetime
from dataclasses import dataclass

DIR_OFFERS = Path("N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH")


@dataclass
class FileSettings:
    directory: str
    output_dir: str
    output_file: str
    sheet_name: str
    header_start: int
    db_file: str = DATABASE_FILE.as_posix()

    def get_filename(self):
        return (
            "#"
            + datetime.datetime.strftime(datetime.datetime.now(), "%Y%m%d")
            + self.output_file
        )

    def get_output_path(self):
        return self.output_dir / (
            "#"
            + datetime.datetime.strftime(datetime.datetime.now(), "%Y%m%d")
            + self.output_file
        )


conf = FileSettings(
    DIR_OFFERS,
    DIR_OFFERS.parent / ".outputs",
    "_Coral_Homes_Offers_Data.xlsx",
    "Offers Data",
    6,
)


def load_previous_data(
    sheet: str = conf.sheet_name, start_row: int = conf.header_start
):
    latest_file_name_pattern = conf.output_file
    matching_files = []
    for f in Path(conf.output_dir).glob("*.*"):
        if re.search(latest_file_name_pattern, f.name):
            print(
                "Match en el fichero",
                f.name,
                "según el patrón",
                latest_file_name_pattern,
            )
            matching_files.append(f)

    sorted_files = sorted([f for f in matching_files], key=os.path.getmtime)
    previous_file = sorted_files[-1]
    return previous_file


load_previous_data()

## Función PIVOT para las tablas resumen y los charts

In [2]:
import duckdb
from pathlib import Path

DATABASE_FILE = Path("N:/CoralHudson/1. AM/8. Wholesale Channel/.code/database.db")

with duckdb.connect(DATABASE_FILE.as_posix()) as db:
    # db.execute("""
    db.sql(
        """
            WITH pivot_data AS (
               PIVOT (select  *, 
                    week(offer_date) as week, 
                    year(offer_date) as year,
               from offers_enriched_table where offer_date > now() - interval 3 months 
           and (commitment_amount is not null or sold_amount is not null))
           ON (year)
,           USING   COUNT(unique_id) as assets, 
                    SUM(total_urs) as total_urs, 
                    SUM(offer_price) as offer_amount, 
                    SUM(commitment_amount) as commitments, 
                    SUM(sold_amount) as sales,
                    SUM(total_proceeds) as total_proceeds,
                    SUM(lsevdec19) as lsev_dec19,
                    SUM(ppa) as ppa,
           GROUP BY week
           )
           select *,
                    round(("2023_total_proceeds" / "2023_ppa" - 1) * 100.0, 2)  || '%' as perf_vs_ppa,
                    round(("2023_total_proceeds" / "2023_lsev_dec19" - 1) * 100.0, 2) || '%' as perf_vs_lsev19,
           from pivot_data;
    """
    ).show()
    # """).df().to_clipboard()

┌───────┬─────────────┬────────────────┬───┬────────────────────┬────────────────────┬─────────────┬────────────────┐
│ week  │ 2023_assets │ 2023_total_urs │ … │  2023_lsev_dec19   │      2023_ppa      │ perf_vs_ppa │ perf_vs_lsev19 │
│ int64 │    int64    │     double     │   │       double       │       double       │   varchar   │    varchar     │
├───────┼─────────────┼────────────────┼───┼────────────────────┼────────────────────┼─────────────┼────────────────┤
│    18 │           2 │           22.0 │ … │ 1612332.3279041096 │          1210758.1 │ -18.23%     │ -38.6%         │
│    19 │           5 │          199.0 │ … │  6011550.821496575 │ 4472445.2700000005 │ -59.11%     │ -69.58%        │
│    20 │           5 │           27.0 │ … │ 2460844.1461869865 │ 1915915.1099999999 │ -8.66%      │ -28.89%        │
│    21 │           4 │           27.0 │ … │  6879029.773816437 │         5682543.28 │ -15.31%     │ -30.04%        │
│    22 │           3 │            3.0 │ … │  4250497.90

### Calcular las distancias dinamicas entre strats

In [ ]:
inicio, distancia = 5, 2
tamaños = [5, 4, 6, 4]

for v in tamaños:
    anterior = v + inicio + distancia
    print("Las coordenadas serían:", "(Inicio)", inicio + idx, v, distancia)

## Leer SQL y partir queries entre comentarios

In [ ]:
import re

query_file = "./queries/strats.sql"

with open(query_file, "r", encoding="utf8") as f:
    queries = re.split("--.+\n", f.read())

for q in queries:
    print("Query:\n", q, sep="===========\n")

with duckdb.connect(DATABASE_FILE.as_posix()) as db:
    pass

## Nueva STOCK QUERY: mas agil y completa

In [20]:
with duckdb.connect(DATABASE_FILE.as_posix()) as db:
    # db.sql("""
    db.execute(
        """
    with base_data as (
    select coalesce(nullif(commercialdev, 0), nullif(jointdev, 0), ur_current)::int as asset_id,
         case
         when label is not null
             then
                 case
                     when publishingchannel_description is null
                         then 'Wholesale'
                     when publishingchannel_description = 'Retail'
                         then 'No longer in Wholesale'
                     else publishingchannel_description
                 end
         else 
             case when publishingchannel_description = 'Wholesale'
               then 'Wholesale - from Retail'
             end
     end as category,
    array_agg(ur_current)::int[] as all_urs,
    count(*) as total_urs,
    count(ur_current) filter (where updatedcategory = 'Sold Assets') as sold_urs,
    sum(saleprice) filter (where updatedcategory = 'Sold Assets') as sale_amount,
    count(ur_current) filter (where updatedcategory = 'Sale Agreed') as comitted_urs,
    sum(commitmentprice) filter (where updatedcategory = 'Sale Agreed') as comittment_amount,
    from master_tape
    group by 1,2
   )
    select b.*,
            case when b.total_urs = b.sold_urs then 'Fully Sold'
               when sold_urs > 1 then 'Partially Sold'
               else 'Remaining Stock'
            end as commercial_status,
            sale_amount + comittment_amount as total_proceeds,
              mode(a.liqrat) as liquidity_rating,
              mode(a.bucket1) as asset_bucket,
           sum(a.lsev_dec19) as lsev_dec19,
           sum(a.capex_uw) as capex_underwriting,
           from base_data b
    cross join unnest(b.all_urs) as c (urs)
    left join allocation_new a on c.urs = a.ur
    where b.category is not null
    group by all
    order by category, asset_bucket
    """
    ).df().to_clipboard()
    # """).show()

# 🐼🧸 POLARS: Testeo de carga de ficheros y tal

In [ ]:
import polars as pl

df = pl.read_excel(
    "N:/CoralHudson/1. AM/8. Wholesale Channel/.outputs/#20230717_WS_Pipeline.xlsx",
    sheet_name="Pipeline 2023",
    xlsx2csv_options={
        "delimiter": ";",
    },
    read_csv_options={
        "separator": ";",
        "infer_schema_length": 10000,
        "skip_rows": 5,
        "ignore_errors": True,
    },
)
df

# 1. WS Stock: creacion de fichero
Usando DuckDB

In [ ]:
db = duckdb.connect()
db.execute(
    f"CREATE VIEW master_tape AS SELECT * FROM parquet_scan('{directorio}/master_tape.parquet')"
)
db.execute(
    f"CREATE VIEW offers_data AS SELECT * FROM parquet_scan('{directorio}/offers.parquet')"
)
db.execute(
    f"CREATE VIEW ws_segregated AS SELECT * FROM parquet_scan('{directorio}/disaggregated_assets.parquet')"
)

In [ ]:
db.execute("select * from master_tape").df().to_clipboard()

In [ ]:
agg_data = db.execute("""
with ranked_offers as (
  select
    o.commercialdev,
    o.offerid,
    o.offerdate,
    o.offerstatus,
    sum(o.offerprice) as offer_total,
    array_agg(o.ur_current) as unique_urs,
    row_number() over (
      partition by o.commercialdev
      order by o.offerdate desc
    ) as row_num
  from offers_data o
  group by 1,2,3,4
),
latest_offers as (
  select * from ranked_offers where row_num = 1
)
select coalesce(nullif(mt.commercialdev, 0), nullif(mt.jointdev,0), mt.ur_current) as asset_id,
            ro.offerid,
            ro.offerdate,
            ro.offerstatus,
            ro.offer_total,
            ro.unique_urs,
            string_agg(distinct city, '|') as city_province, 
            string_agg(distinct direccion_territorial, '|') as dts, 
            count(*) as total_urs,
            count(*) filter (where updatedcategory = 'Sold Assets') as sold_assets, 
            sum(lsev_dec19) filter (where updatedcategory = 'Sold Assets') as sold_lsev, 
            sum(ppa) filter (where updatedcategory = 'Sold Assets') as sold_ppa, 
            sum(nbv) filter (where updatedcategory = 'Sold Assets') as sold_nbv, 
            count(*) filter (where updatedcategory = 'Sale Agreed') as committed_assets, 
            sum(lsev_dec19) filter (where updatedcategory = 'Sale Agreed') as committed_lsev, 
            sum(ppa) filter (where updatedcategory = 'Sale Agreed') as committed_ppa, 
            sum(nbv) filter (where updatedcategory = 'Sale Agreed') as committed_nbv, 
            count(*) filter (where updatedcategory = 'Remaining Stock') as remaining_assets, 
            sum(lsev_dec19) filter (where updatedcategory = 'Remaining Stock') as remaining_lsev, 
            sum(ppa) filter (where updatedcategory = 'Remaining Stock') as remaining_ppa, 
            sum(nbv) filter (where updatedcategory = 'Remaining Stock') as remaining_nbv, 
                from master_tape mt
            left join
              latest_offers ro on mt.commercialdev = ro.commercialdev
            where
                label is not null
            group by 1,2,3,4,5,6""").df()

In [ ]:
agg_data.to_clipboard()

## Esto es la BIBLIA de cómo tratar lo mayorista, agarra casi todos los casos
Además, añadir los 500 y pico desagregados y ver si actualmente coinciden (es decir, están ya considerados mayorista)

- Cuando 2 es Mayorista: 
     - Cuando 1 está vacío: Mayorista (vendido)
     - Cuando 1 es Retail: Retail (fue mayorista)
     - Cuando 1 es Mayorista: Mayorista
- Cuando 2 está vacío: 
     - Cuando 1 es Mayorista: Mayorista (cambiado desde retail inicial)

In [ ]:
db.execute("select * from ws_segregated").df()

In [1]:
import duckdb

with duckdb.connect("C:/Users/amontes/Desktop/database.db") as db:
    with open("./queries/stock_data.sql", encoding="utf8") as fq:
        query = fq.read()
        # db.execute(query).df().to_clipboard()
        db.sql(query).df().to_clipboard()

## Segregaciones y ventas al menudeo de mayoristas, crear fichero parquet

In [ ]:
fichero = Path("C:/Users/amontes/Desktop/").glob("[!$~]*.xlsx")
(
    pd.read_excel(next(fichero))
    .filter(["UNIDAD_REGISTRAL", "ESTRATEGIA"], axis=1)
    .query("ESTRATEGIA.notnull()")
    .to_parquet(
        "//EURFL01/advisors.hal/non-hudson/Coral Homes/CoralHudson/6. Stock/Parquet/disaggregated_assets.parquet",
        index=False,
    )
)

# Acceso al inbox

## Explorar el Inbox de forma adecuada

In [ ]:
import datetime
import re
import win32com.client
import glob
from pathlib import Path

outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.GetDefaultFolder(6)  # "6" refers to the inbox

number_of_months = 4
subject_pattern = r"\d{6,8} Pipe 20"
filename_pattern = r"^\d{6,8} Pipe 20"
message_count = 0
duplicados = 0
guardados = 0
ficheros_existentes = [
    Path(f).name
    for f in glob.glob("N:/CoralHudson/1. AM/8. Wholesale Channel/WS PIPE*/[!~$]*.*")
]
extensions = [".xlsx", ".xls", ".xlsb", ".xlsm"]
print(ficheros_existentes)


# Filtrado por fecha
time_range = datetime.date.today() - datetime.timedelta(30 * int(number_of_months))
time_string = time_range.strftime("%m/%d/%Y")
print(f"Buscando mensajes recibidos desdes el { time_range.strftime('%d/%m/%Y')}")
sFilter = f"[ReceivedTime] >= '{time_string}'"
print(sFilter)
messages = inbox.Items.Restrict(sFilter)
print(messages.count)

for message in messages:
    if re.search(subject_pattern.lower(), message.Subject.lower()):
        print(message.Subject, ":received on:", message.ReceivedTime)
        attachments = message.Attachments
        for attachment in attachments:
            if re.search(filename_pattern.lower(), attachment.FileName.lower()) and any(
                attachment.FileName.endswith(ext) for ext in extensions
            ):
                # Filtrar aquellos que ya estan encontrados
                if attachment.FileName in ficheros_existentes:
                    print("\tAttachment ignored. Already in folder.")
                    duplicados += 1
                else:
                    print("\tAttachment found:", attachment.FileName)
                    guardados += 1

print(f"Guardados {guardados} ficheros en el directorio base.")
print(f"Omitidos {duplicados} ficheros por ya encontrarse en la carpeta.")

In [ ]:
from rich.console import Console

console = Console()


def retrieve_attachments(
    subject_pattern: str,
    filename_pattern: str,
    extensions: list,
    file_type: str,
    top_level_dir: str,
    number_of_months: int,
):
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    inbox = outlook.GetDefaultFolder(6)  # "6" refers to the inbox

    # Conseguir todos los ficheros del directorio
    # Listar todos los ficheros del directorio de destino, y filtrar contra los encontrados
    console.print(f"Checking existing {file_type} files...")
    match top_level_dir, file_type:
        case "coralhudson", "pipe":
            base_dir = Path("N:/CoralHudson/1. AM/8. Wholesale Channel/")
            ficheros_existentes = [
                Path(f).name
                for f in glob.glob(
                    "N:/CoralHudson/1. AM/8. Wholesale Channel/WS PIPE*/[!~$]*.*"
                )
            ]
        case "currentdir", "pipe":
            base_dir = Path("./_attachments/pipe_files/")
            ficheros_existentes = [f.name for f in base_dir.rglob("*.*")]
        case "coralhudson", "offers":
            base_dir = Path(
                "N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/"
            )
            ficheros_existentes = [
                Path(f).name
                for f in glob.glob(
                    base_dir.as_posix() + "/Ofertas recibidas SVH/2023/**/[!~$]*.*"
                )
            ]
        case "currentdir", "offers":
            base_dir = Path("./_attachments/offer_files/")
            ficheros_existentes = [f.name for f in base_dir.rglob("*.*")]

    # Filtrado por fecha
    time_range = datetime.date.today() - datetime.timedelta(
        days=30 * int(number_of_months)
    )
    time_string = time_range.strftime("%m/%d/%Y")
    print(f"Buscando mensajes recibidos desdes el { time_range.strftime('%d/%m/%Y')}")
    sFilter = f"[ReceivedTime] >= '{time_string}'"
    print(sFilter)
    messages = inbox.Items.Restrict(sFilter)
    print(messages.count)
    if messages.count == 0:
        return "No more, stop, there are no messages"

    message_count = 0
    duplicados = 0
    guardados = 0
    for message in messages:
        if re.search(subject_pattern.lower(), message.Subject.lower()):
            message_count += 1
            received_time = message.ReceivedTime
            console.print(
                f"[{message_count}] Found email:",
                message.Subject,
                "received on",
                received_time,
            )
            attachments = message.Attachments
            for attachment in attachments:
                if re.search(
                    filename_pattern.lower(), attachment.FileName.lower()
                ) and any(attachment.FileName.endswith(ext) for ext in extensions):
                    # Filtrar aquellos que ya estan encontrados
                    if attachment.FileName in ficheros_existentes:
                        console.print("\tAttachment ignored. Already in folder.")
                        duplicados += 1
                    else:
                        console.print(
                            "\tAttachment found:", attachment.FileName, style="bold red"
                        )
                        guardados += 1
                        # Get the email's received time and format it as a directory path
                        year_folder = received_time.strftime("%Y")
                        day_folder = received_time.strftime("%Y%m%d")

                        match file_type:
                            case "pipe":
                                full_dir = base_dir / f"WS PIPE {year_folder}"
                            case "offers":
                                full_dir = base_dir / year_folder / day_folder

                        # Create the directory if it doesn't exist
                        full_dir.mkdir(parents=True, exist_ok=True)

                        # Save the attachment in the directory
                        console.print(
                            f"\tAttempting to save down to {(full_dir / attachment.FileName).absolute()}"
                        )
                        attachment.SaveASFile(
                            str((full_dir / attachment.FileName).absolute())
                        )
    console.print(f"Guardados {guardados} ficheros en el directorio base: {base_dir}.")
    console.print(f"Omitidos {duplicados} ficheros por ya encontrarse en la carpeta.")


retrieve_attachments(
    subject_pattern, filename_pattern, extensions, "pipe", "coralhudson", 5
)

## TIMEDELTAS: Calcular correctamente

In [ ]:
import datetime

seis_meses_atras = datetime.datetime.now() - datetime.timedelta(30 * 6)
seis_meses_atras

In [ ]:
from pathlib import Path

direc = [f.name for f in Path("./_attachments").rglob("*.*")]
nombre_filtrado = "191104_Wip Calle santa rosa_el escotrial_6079527.xlsx"

nombre_filtrado in direc

In [ ]:
import os
import glob
import pandas as pd
from pathlib import Path

base_dir = Path("N:/CoralHudson/1. AM/8. Wholesale Channel/")
# ficheros_existentes = [f for f in os.walk(base_dir / "WS PIPE 2023")]
ficheros_existentes = [
    Path(f).name
    for f in glob.glob(base_dir.as_posix() + "/Ofertas recibidas SVH/2023/**/*.*")
]

In [ ]:
pd.DataFrame({"ficheros": ficheros_existentes}).to_clipboard()

# Obtener agregados por promo de valores, conteos

In [ ]:
import pandas as pd
from pathlib import Path
import os

file = pd.read_parquet(
    "//EURFL01/advisors.hal/non-hudson/Coral Homes/CoralHudson/6. Stock/Parquet/master_tape.parquet"
)
file.columns

In [ ]:
dict_agg = {"lsev_dec19": "sum", "ppa": "sum", "nbv": "sum", "ur_current": "count"}

(file.groupby(["commercialdev", "updatedcategory"]).aggregate(dict_agg))

In [ ]:
def include_values(df: pd.DataFrame, datapoints_to_include: str) -> pd.DataFrame:
    datapoints_dict = json.loads(datapoints_to_include)
    for file_info in datapoints_dict.keys():
        # Extract only the columns needed
        file = file[[file_info["primary_key"]] + file_info["datapoints"]]
        # Merge the data into df_to_modify
        df_to_modify = pd.merge(
            df_to_modify, file, on=file_info["primary_key"], how="left"
        )

    return df_to_modify

# Ofertas: busqueda de etiquetas y posiciones relativas a las mismas
Obtener datos según rangos con nombre propio: en este caso "Print Area"

In [ ]:
from pathlib import Path
from rich.console import Console
import pendulum as pdl
import openpyxl
import os
import json
import datetime
import re


def load_json_config(file):
    """
    Loads a JSON file containing external config data, like cell addresses and their corresponding output labels.
    """
    with open(file, "r", encoding="utf8") as f:
        return json.load(f)


console = Console()

files = []

In [ ]:
def extract_cell_values(
    file: str, search_strings: dict, columns_dict: dict, table_sheet: str = "SAP"
):
    """
    Opens an Excel file and converts the worksheet to a dictionary. It then looks for
    specific strings, moves to the relative cell addresses based on both x and y offsets,
    and extracts the value at that location. Also extracts unique values from specified
    columns in a table on another sheet.
    Includes the file name in the returned data.
    """
    try:
        workbook = openpyxl.load_workbook(file, read_only=True, data_only=True)
    except Exception as e:
        console.print(f"Error when loading file {file}. Details: {e}")
        data = {}
        data["read_status"] = "Fail"
        data["read_details"] = str(e)
        data["full_path"] = os.path.abspath(file)
        data["file_name"] = os.path.basename(file)
        return data
    sheet = workbook["FICHA"]
    data = {label: None for label in search_strings.keys()}
    data["full_path"] = os.path.abspath(file)
    data["file_name"] = os.path.basename(file)
    data["read_status"] = "Success"
    data["read_details"] = None

    # Convert the worksheet to a dictionary for faster searching
    sheet_dict = {
        (cell.row, cell.column): cell.value
        for row in sheet.iter_rows()
        for cell in row
        if cell.value is not None
    }

    for (cell_row, cell_col), cell_value in sheet_dict.items():
        for label, [regex, offset_y, offset_x] in search_strings.items():
            if re.match(
                regex, str(cell_value), re.IGNORECASE
            ):  # compare values using the regular expression
                try:
                    target_cell_address = (
                        cell_row - offset_y,
                        cell_col + offset_x,
                    )
                    target_cell_value = sheet_dict.get(target_cell_address)
                    if target_cell_value is not None:
                        data[label] = target_cell_value
                        break
                except IndexError:
                    continue
                else:
                    break
            else:
                continue

    # Load the JSON file for table columns and process table columns if specified
    if columns_dict and table_sheet:
        try:
            table_sheet = workbook[table_sheet]
            for col in columns_dict:
                # Assume the first row contains headers
                header_row = next(
                    table_sheet.iter_rows(min_row=1, max_row=1, values_only=True)
                )
                # Get index of the target column based on header
                try:
                    col_idx = header_row.index(col)
                except ValueError:
                    console.print(
                        f'Warning: Column header "{col}" not found in sheet. Skipping this column.'
                    )
                    continue
                # Extract unique values from the column
                column_values = [
                    cell[col_idx]
                    if not isinstance(cell[col_idx], str) or not cell[col_idx].isdigit()
                    else int(cell[col_idx].lstrip("0"))
                    for cell in table_sheet.iter_rows(min_row=2, values_only=True)
                ]
                # Remove duplicates by converting to a set, after filtering out the null values
                unique_values = set(filter(None, column_values))
                # If there's only one unique value, unpack it from the set
                if len(unique_values) == 1:
                    unique_values = unique_values.pop()
                    if isinstance(unique_values, str) and unique_values.isdigit():
                        unique_values = int(
                            unique_values.lstrip("0")
                        )  # Remove leading zeros for numeric strings
                else:
                    # Convert to list and sort to make output more predictable
                    try:
                        unique_values = sorted(unique_values)
                    except TypeError as e:
                        console.print(f"Error en fichero {file} debido a {e}")
                        data["read_status"] = "Warning"
                        data["read_details"] = str(e)
                        continue
                # Add the unique values to the output
                data[columns_dict.get(col)] = unique_values if unique_values else None
        except KeyError as e:
            console.print(f"Error encontrado: {e}\nFichero causante: {file}")
            data["read_status"] = "Warning"
            data["read_details"] = str(e)

    # Cleanup of bad data or strings
    for label, value in data.items():
        if label == "contract_deposit" and value == "-":
            data[label] = 0
        elif label == "client_description" and value == "NOMBRE":
            data[label] = None
        elif label == "offer_date":
            try:
                if isinstance(value, str):
                    value = pdl.parse(value, strict=False).to_date_string()
                elif isinstance(value, datetime.datetime):
                    data[label] = value
            except Exception as e:
                console.print(f"Error al identificar la fecha de la oferta: >> {e}")
                if data["full_path"].startswith("\\\\EURFL01"):
                    posible_fecha = re.findall("\d{6,8}", data["full_path"])[0]
                    data[label] = datetime.datetime.strptime(posible_fecha, "%Y%m%d")
                else:
                    data[label] = datetime.datetime.strptime(
                        "".join(data["full_path"].split("\\")[-4:-1]), "%Y%m%d"
                    )
        else:
            data[label] = value

    return data

In [ ]:
cell_addresses = load_json_config("./conf/cell_addresses.json")
sap_mapping_file = load_json_config("./conf/sap_columns_mapping.json")

data = []

for file in files[:3]:
    data.append(extract_cell_values(file, cell_addresses, sap_mapping_file))

In [ ]:
file = files[9]
print("Opening file:\n", file)
test_wb = openpyxl.load_workbook(file, read_only=True, data_only=True)
indice_hoja = list(
    filter(lambda z: re.match("xd", z, flags=re.IGNORECASE), test_wb.sheetnames)
)
indice_hoja = indice_hoja.pop() if indice_hoja else "FICHA"
test_wb[indice_hoja]

In [ ]:
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string

test_wb = openpyxl.load_workbook(files[3], read_only=True, data_only=True)
sheet = test_wb["FICHA"]
primera_celda, ultima_celda = sheet.calculate_dimension().split(":")
last_row = coordinate_from_string(ultima_celda)[1]
last_column = column_index_from_string(coordinate_from_string(ultima_celda)[0])
print("La ultima fila es", last_row, "y la ultima columna es", last_column)

In [ ]:
for f in Path("./_attachments/offer_files/").rglob("[!$^][0-9]*.xlsx"):
    print(f"Contenidos del fichero {f.stem}:", extract_cell_values(f, ss), sep="\n")

## Encontrar valores en la pestaña "SAP" y aglutinar conteos de cada categoría (tipo de activo) y únicos/primero (descripción)

In [ ]:
def extract_cell_values(
    file, search_strings, table_columns: dict, table_sheet: str = "SAP"
):
    """
    Opens an Excel file and converts the worksheet to a dictionary. It then looks for
    specific strings, moves to the relative cell addresses based on both x and y offsets,
    and extracts the value at that location. Also extracts unique values from specified
    columns in a table on another sheet.
    Includes the file name in the returned data.
    """
    workbook = openpyxl.load_workbook(file, read_only=True, data_only=True)
    sheet = workbook.active
    data = {label: None for label in search_strings.keys()}
    data["file_name"] = os.path.basename(file)

    # Convert the worksheet to a dictionary for faster searching
    sheet_dict = {
        (cell.row, cell.column): cell.value
        for row in sheet.iter_rows()
        for cell in row
        if cell.value is not None
    }

    for (cell_row, cell_col), cell_value in sheet_dict.items():
        for label, (string, offset_y, offset_x) in search_strings.items():
            if cell_value == string:
                target_cell_address = (cell_row + offset_y, cell_col + offset_x)
                data[label] = sheet_dict.get(target_cell_address)

    # Process table columns if specified
    if table_columns and table_sheet:
        table_sheet = workbook[table_sheet]
        for col in table_columns.keys():
            # Assume the first row contains headers
            header_row = next(
                table_sheet.iter_rows(min_row=1, max_row=1, values_only=True)
            )
            # Get index of the target column based on header
            try:
                col_idx = header_row.index(col)
            except ValueError:
                print(
                    f'Warning: Column header "{col}" not found in sheet. Skipping this column.'
                )
                continue
            # Extract unique values from the column
            column_values = [
                cell[col_idx]
                for cell in table_sheet.iter_rows(min_row=2, values_only=True)
            ]
            unique_values = set(
                filter(None, column_values)
            )  # Remove duplicates by converting to a set, after filtering out the null values
            # If there's only one unique value, unpack it from the set
            if len(unique_values) == 1:
                unique_values = unique_values.pop()
            # Add the unique values to the output
            data[table_columns.get(col)] = unique_values

    return data

In [ ]:
label_mapping = {
    "Tipo de inmueble": "asset_type",
    "Dirección": "address",
    "Emplazamiento": "asset_location",
}

print(
    "Contenidos del primer fichero:",
    extract_cell_values(file_2, ss, label_mapping),
    sep="\n====\n",
)

In [ ]:
valores = ["-", "COOODE", "-"]
v1 = set(valores)
v1.discard("-")
v1

## Funcion Cargar direcciones desde JSON no funciona, vamos a arreglarla

In [ ]:
import json


def load_cell_addresses(file):
    """
    Loads a JSON file containing cell addresses and their corresponding output labels.
    """
    with open(file, "r") as f:
        return json.load(f)


load_cell_addresses("./const/cell_addresses.json")

# 🧪📗 OPENPYXL: Juguemos

## Listar los estilos creados "nombrados"

In [ ]:
import openpyxl

fichero = openpyxl.load_workbook("./_output/#20230703_Wholesale_Stock.xlsx")
hoja = fichero.active
hoja.parent.named_styles

## Usar regex para buscar las hojas

In [ ]:
import re
import openpyxl

# fichero = openpyxl.load_workbook("N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2023/20230210/20230210 OF 5525400 PROMO 60466091 Nave industrial en Arakundiaga.xlsx")
# fichero = openpyxl.load_workbook("N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2023/20230707/20230707_OF_5549328_UR_60460925_SUELO_AV.BENEMIS 29 PILES_VALENCIA.xlsx")
fichero = openpyxl.load_workbook(
    "//EURFL01/advisors.hal/non-hudson/Coral Homes/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2023/20230215/20230215_ OF  5526017  UR  60473587 Nave  Cl Aracena. Huelva.xlsx"
)
candidatas = []
for sh in fichero.sheetnames:
    if re.search(r"\b(?:SAP|^Oferta)\b", sh, re.IGNORECASE):
        print("Match:", sh)
regex = re.compile(r"\b(?:SAP|^Oferta)\b", re.IGNORECASE)
list(filter(regex.search, fichero.sheetnames))[0]

## Ignorar hojas que contengan algún tipo de imagen

In [ ]:
fichero_con_imagen = "//EURFL01/advisors.hal/non-hudson/Coral Homes/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2023/20230424/20230424 OF_5537626_Promo_6081831_Local_Ctra. de Madrid_ZARAGOZA.xlsx"
fichero_sin_imagen = "//EURFL01/advisors.hal/non-hudson/Coral Homes/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2023/20230215/20230215_ OF  5526017  UR  60473587 Nave  Cl Aracena. Huelva.xlsx"
fichero = openpyxl.load_workbook(fichero_sin_imagen)
for sh in fichero.sheetnames:
    if re.search(r"\b(?:SAP|^Oferta)\b", sh, re.IGNORECASE) and not fichero[sh]._images:
        print("Match:", sh)
regex = re.compile(r"\b(?:SAP|^Oferta)\b", re.IGNORECASE)
list(filter(regex.search, fichero.sheetnames))

fichero["Oferta"]._images

In [ ]:
hojas = [1, 2, 3]
hojas.index(1)

## Detectar la ultima columna y convertirla a coordenadas Excel

In [ ]:
from openpyxl.utils import get_column_letter

last_column_as_letter = get_column_letter(1)
last_column_as_letter

# 🧲 DATACLASSES como reemplazo de argumentos en funciones

In [ ]:
from dataclasses import dataclass
import pandas as pd


@dataclass
class SetOfArgs:
    title: str
    datapack: list
    sheet: int


def admits_dc(title: str, datapack: list, sheet: int) -> str:
    print(title, datapack, sheet)


config = SetOfArgs("kids", ["are", "not", "for", "sale"], 1)

admits_dc(**parametros.__dict__)

config.sheet

# Generar un UUID segun nombre de fichero

In [ ]:
import uuid

# This is a known namespace for DNS. You could also define your own namespace UUID.
namespace = uuid.NAMESPACE_DNS

unique_string = "C:/Users/amontes/Downloads/nvim-win64.zip"

uuid5 = uuid.uuid5(namespace, unique_string)

print(uuid5)

In [ ]:
import duckdb

with open("./queries/offers_query.sql", encoding="utf8") as sql_file:
    query = sql_file.read()

with duckdb.connect("./basedatos_wholesale.db") as db:
    data = db.execute(query).df()

data.to_clipboard()

In [ ]:
previous = None
if previous:
    print("None NO es igual que false")
else:
    print("None es igual que false")

Esto es una prueba de regex normal

In [ ]:
import re

cell_address_file = "./const/cell_addresses.json"
sap_mapping_file = "./const/sap_columns_mapping.json"


def load_json_config(file):
    """
    Loads a JSON file containing external config data, like cell addresses and their corresponding output labels.
    """
    with open(file, "r", encoding="utf8") as f:
        return json.load(f)


cell_addresses = load_json_config(cell_address_file)
label_pattern = cell_addresses.get("offer_id")[0]
re.match(label_pattern, "Núm..\nOFERTA")

UNION de tablas en string

In [ ]:
import duckdb

all_duckdb_tables = ["ws_current_offers", "ws_hist_offers"]

with duckdb.connect("./basedatos_wholesale.db") as db:
    df = db.execute(
        " UNION ".join(["select * from " + t for t in all_duckdb_tables])
    ).df()

df.to_clipboard()

## Obtener valores de URs individuales y su suma posterior

In [ ]:
import pandas as pd

columns_to_use = [
    "offer_id",
    "total_urs",
    "ppa",
    "lsev_dec19",
    "commercialdev",
    "jointdev",
    "unique_urs",
]

# Datos
datos_excel = pd.read_excel(
    "C:/Users/amontes/Projects/wholesale/_output/#20230613_Coral_Homes_Offers_Data.xlsx",
    skiprows=4,
    usecols=columns_to_use,
)

In [ ]:
datos_excel

In [ ]:
def sum_corresponding_values(
    df, column, con, lookup_source, column_id_source, value_source
):
    # Step 1: Split the column into separate rows for each ID
    df = (
        df[column]
        .fillna(0)
        .astype(str)
        .str.replace("[()]", "", regex=True)
        .str.replace("|", ",")
        .str.split(",")
        .explode()
    )
    # Step 2: Load the DataFrame into DuckDB
    con.register("df", df.reset_index())
    # Step 3: Join the DataFrame with the table containing the corresponding values for each ID
    # Step 4: Group by the original index and sum the corresponding values
    query = """SELECT df.index, count(*), sum(v.{}) as summed
    FROM df
    LEFT JOIN {} v ON df.{} = v.{}
    GROUP BY 1
    """.format(value_source, lookup_source, column, column_id_source)
    result = con.execute(query).df()
    result.to_clipboard()
    # Step 5: Return a Series with the sum of corresponding values for each original index
    return result.set_index("index").iloc[:, :2]


datos_excel[["count_urs", "sum"]] = datos_excel.pipe(
    sum_corresponding_values,
    "unique_urs",
    db,
    "master_tape",
    "ur_current",
    "lsev_dec19",
)

In [ ]:
datos_excel

## Conseguir el año mes y dia de la oferta, segun el nombre y ruta del fichero

In [ ]:
from datetime import datetime

fich = "//EURFL01/advisors.hal/non-hudson/Coral Homes/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2019/20191105/191105_AV MADRID_Suelo_El molar_60502631.xlsx"

datetime.strptime(fich.split("/")[-2], "%Y%m%d")

Usar otras librerías como Pendulum o Arrow

In [ ]:
import pendulum as pdl

pdl.set_locale("es")

fechas_tipo = [
    "30/112022",
    "30112022",
    "29.11.2022",
    "24.01.2023",
    "17 Noviembre de 2022",
    "14 Diciembre de 2022",
    "11.11.2022",
    "09.11.2022",
    "03.02.2023",
    "01.03.2023",
]

for date in fechas_tipo:
    try:
        print(pdl.parse(date, strict=False).to_date_string())
    except Exception as e:
        print(e)

In [ ]:
import re

carpeta = "//EURFL01/advisors.hal/non-hudson/Coral Homes/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2022/20221223 ---/221223, Cl. Olzinelles N.64, Barcelona, 6073845 - NIA INVEST SL_rev - 5516178.xlsx"
re.findall("\d{6,8}", carpeta)[0]

## Combinar dos columnas con valores nulos en cada, a través del mismo dataframe (concatenacion de metodos)

In [ ]:
import pandas as pd
import datetime as dt

df1 = pd.DataFrame(
    {
        "dato": [1, 2, 3],
        "mata": [0, None, None],
        "relato": [None, 4, 5],
        "fechas": [
            dt.datetime.now(),
            dt.datetime(2012, 10, 5),
            dt.datetime(2022, 4, 30, 11, 30),
        ],
    }
)
(
    df1.assign(
        combinado=lambda df_: df_["mata"].fillna(df_.relato),
        otro_mas=lambda df_: df_["mata"].fillna(df_.relato),
    ).sort_values(by=["fechas"], ascending=False)
)

# Inspeccionar los totales de ficheros de ofertas, por tipo de extensión

In [ ]:
from pathlib import Path
import pandas as pd

directory = "//EURFL01/advisors.hal/non-hudson/Coral Homes/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH"
ficheros = Path(directory).rglob("[!$^]*.*")

In [ ]:
listado_ficheros_ofertas = [(f.stem, f.suffix) for f in list(ficheros)]

In [ ]:
(
    pd.DataFrame(listado_ficheros_ofertas, columns=["filename", "extension"])
    .groupby("extension")
    .count()
)

# Carriage Return (imprimir linea sobre linea)

In [ ]:
import time
from pathlib import Path

data = Path.cwd()
ficheros_leidos = 0

datafiles2 = list(Path("C:/Users/amontes/Downloads/").glob("*.*"))
num_ficheros = len(datafiles2)

for i in datafiles2:
    ficheros_leidos += 1
    print(
        ficheros_leidos, end=f" completado, quedan {num_ficheros - ficheros_leidos}\r"
    )
    time.sleep(0.05)

print("Okay terminado")

In [ ]:
import datetime

datos = None

isinstance(datos, (datetime.datetime, None))

# Fichero Pipe: abrir, completar con datos, crear dashboard automático

In [34]:
from pathlib import Path
import re
import datetime

template_file = "N:/CoralHudson/1. AM/8. Wholesale Channel/WS PIPE 2023"
# Define the format for parsing the date from the file name
date_format = "%Y_%d_%B"

ficheros_pipe = Path(template_file).glob("*.xlsx")
for f in ficheros_pipe:
    try:
        # Parse the date from the file name using datetime.strptime
        parsed_date = datetime.datetime.strptime(f.name, f"%*_{date_format}.xlsx")
        print(parsed_date.strftime("%d/%m/%Y"))  # Output: "03/05/2023"
    except ValueError as e:
        print("Date not found in the file name.", e)

Date not found in the file name. '%' is a bad directive in format '%*_%Y_%d_%B.xlsx'

Date not found in the file name. '%' is a bad directive in format '%*_%Y_%d_%B.xlsx'

Date not found in the file name. '%' is a bad directive in format '%*_%Y_%d_%B.xlsx'

Date not found in the file name. '%' is a bad directive in format '%*_%Y_%d_%B.xlsx'

Date not found in the file name. '%' is a bad directive in format '%*_%Y_%d_%B.xlsx'

...

Date not found in the file name. '%' is a bad directive in format '%*_%Y_%d_%B.xlsx'

Date not found in the file name. '%' is a bad directive in format '%*_%Y_%d_%B.xlsx'

Date not found in the file name. '%' is a bad directive in format '%*_%Y_%d_%B.xlsx'

Date not found in the file name. '%' is a bad directive in format '%*_%Y_%d_%B.xlsx'

Date not found in the file name. '%' is a bad directive in format '%*_%Y_%d_%B.xlsx'

In [28]:
datetime.datetime.strptime("StupidFile_2023_03_May.xlsx", date_format)

ValueError: time data 'StupidFile_2023_03_May.xlsx' does not match format '_%Y_%d_%B.xlsx'

# Switch Case Simultáneo (2 o más variables)

In [ ]:
var1, var2 = "not ok", 2

match var1, var2:
    case "ok", 1:
        print("Lo agarra!")
    case "not ok", 2:
        print("También lo agarra!")
    case "fuck you", 3:
        print("que alguien pare a este hombre!!")
    case _:
        print("Errm esto no era lo que habiamos hablao")

# OFERTAS: Sólo aquellas que NO existan

In [10]:
from conf.functions import find_files_included
from conf.settings import offersconf as conf

files = find_files_included(conf.directory, "2023")

Scanning offers directory N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2023...

In [26]:
# OK it works for now, lets compare against the path on the sql table

import duckdb
import os

with duckdb.connect(conf.db_file) as db:
    x = db.sql("""select distinct(full_path) from ws_current_offers""").fetchall()

x = {os.path.abspath(n[0]) for n in x}
files = {os.path.abspath(f) for f in files}

list(files.symmetric_difference(x))

['N:\\CoralHudson\\1. AM\\8. Wholesale Channel\\Ofertas recibidas SVH\\2023\\20230920\\20230920_OF 5551195_UR_60453075_SUELO_PJE MAS CAN FALET_SANTA PAU.xlsx']

In [27]:
empty_list = []
if empty_list:
    print("Tiene contenido?")
else:
    print("No tiene contenido?")

No tiene contenido?


# GLOB: Evitar ficheros temporales

In [5]:
from pathlib import Path

dirfil = Path("N:/CoralHudson/1. AM/8. Wholesale Channel/WS PIPE 2023/")
files = [
    p
    for p in dirfil.rglob("*")
    if p.suffix in [".xlsx", ".xls"] and not p.name.startswith("~")
]
files

[WindowsPath('N:/CoralHudson/1. AM/8. Wholesale Channel/WS PIPE 2023/230822 Pipe 2023 - Formalización Coral Comercial V1.xlsx'),
 WindowsPath('N:/CoralHudson/1. AM/8. Wholesale Channel/WS PIPE 2023/230829 Pipe 2023 -Formalización Coral Comercial V1.xlsx'),
 WindowsPath('N:/CoralHudson/1. AM/8. Wholesale Channel/WS PIPE 2023/Copy of WS_Pipe_2023_22_November.xlsx'),
 WindowsPath('N:/CoralHudson/1. AM/8. Wholesale Channel/WS PIPE 2023/WS_Pipe_2023_01_August.xlsx'),
 WindowsPath('N:/CoralHudson/1. AM/8. Wholesale Channel/WS PIPE 2023/WS_Pipe_2023_03_May.xlsx'),
 WindowsPath('N:/CoralHudson/1. AM/8. Wholesale Channel/WS PIPE 2023/WS_Pipe_2023_03_October.xlsx'),
 WindowsPath('N:/CoralHudson/1. AM/8. Wholesale Channel/WS PIPE 2023/WS_Pipe_2023_04_April.xlsx'),
 WindowsPath('N:/CoralHudson/1. AM/8. Wholesale Channel/WS PIPE 2023/WS_Pipe_2023_05_December.xlsx'),
 WindowsPath('N:/CoralHudson/1. AM/8. Wholesale Channel/WS PIPE 2023/WS_Pipe_2023_05_September.xlsx'),
 WindowsPath('N:/CoralHudson/1.

# 👨‍🏭 FIX 2023-2019 OFFERS LOADING

In [14]:
import polars as pl

df = pl.read_csv("debug_ddb_table.csv", ignore_errors=True, null_values=["-"])

In [20]:
df.filter(~pl.col("web_price").str.contains(r"[\d\.,]"))

,offer_id,offer_date,offer_price,appraisal_price,sap_price,web_price,delegate,segment,buyer_meeting,listed_y_n,rollup_y_n,dossier,land_area,buildable_area,main_use,planning_works,land_readjustment_project,urbanization_project,urban_works,commercial_status,legal_status,possession_status,client_name,client_description,client_tax_id,client_phone,client_email,client_address,due_diligence,contract_deposit,public_deed,conditions_details,offer_description_long,servihabitat_opinion,servihabitat_recommendation,offer_lead_type,offer_lead_id,full_path,file_name,read_status,read_details,commercialdev,jointdev,unique_urs,asset_type,address,asset_location
i64,str,str,f64,f64,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1379,"""5480102""",null,1e6,1.382767e6,"""0""","""A Consultar""","""ESTRELLA RODRI…",null,null,null,null,null,"""370""",1069.0,"""RESI""","""NA""","""NA""","""NA""","""NA""","""NO PUESTO A LA…",null,"""VERTICAL""",null,null,null,null,null,null,null,0.1,0.9,"""Contrato priva…","""Edificio para …",null,"""CONTRAOFERTAR""","""SVH""",null,"""N:\CoralHudson…","""220211_ 607952…","""Success""",null,"""6079529""",null,"""60502708""","""EDIFICIO VIVIE…","""CL. FERIA, N.1…","""PENDIENTE DE E…"
1465,"""5472183""",null,1.153841e6,2.2474e6,"""0""","""A consultar""","""Carlos Revenga…",null,null,null,null,null,"""15620""",6931.9217,"""RESI""","""NA""","""NA""","""NA""","""no""","""DISPONIBLE COM…",null,"""PARTE INDIVISA…",null,null,null,null,null,null,null,null,null,"""condicionado a…","""Nos encontramo…",null,"""VENDER""","""API""",null,"""N:\CoralHudson…","""220401_MurgaPr…","""Success""",null,"""[6072835, 6072…","""6072836""","""[60456492, 604…","""['EDIFICIO OFI…","""['POLIGONO IND…","""['POL. IND. DE…"
1468,"""5472227""",null,225000.0,355160.0,"""0""","""A consultar ""","""Fernando Ibañe…",null,null,null,null,null,null,327.0,"""RESI""","""NA""","""NA""","""NA""","""NA""","""DISPONIBLE COM…",null,"""LIBRE CON POSE…",null,null,null,null,null,null,null,0.1,0.9,"""10% arras a lo…","""Obra en curso …",null,"""VENDER""","""API""",null,"""N:\CoralHudson…","""220404_WIP_ Pr…","""Success""",null,"""6076217""","""6113772""","""[60478868, 604…","""['LOCAL', 'VIV…","""['CL. JABONERI…","""VENTA"""
1469,"""5472543""",null,4e6,4.49008e6,"""0""","""No publicado""","""JORGE TARIFA E…",null,null,null,null,null,null,3020.09,"""RESI""","""NA""","""NA""","""NA""","""NA""","""DISPONIBLE COM…",null,"""REVISADA ENTRE…",null,null,null,null,null,null,"""SI""",0.3,0.7,"""CP final de ab…","""Promoción de 1…",null,"""VENDER""","""API""",null,"""N:\CoralHudson…","""220404_WIP_607…","""Success""",null,"""6073011""","""6073012""","""[60457215, 604…","""VIVIENDA""","""['PLAN PARCIAL…","""VENTA"""
1471,"""5460013""",null,250000.0,592126.89,"""0""","""A consultar""","""Sara Saigí Bar…",null,null,null,null,null,null,578.79,"""RESI""","""NA""","""NA""","""NA""","""NA""","""DISPONIBLE COM…",null,"""LIBRE CON POSE…",null,null,null,null,null,null,null,null,1.0,"""Forma de Pago:…","""Edificio pluri…",null,"""VENDER""","""API""",null,"""N:\CoralHudson…","""220406_WIP_ 60…","""Success""",null,"""6075385""","""6075386""","""[60473016, 604…","""['LOCAL', 'PLA…","""['CTRA. GENERA…",null
1472,"""5460013""",null,525000.0,583783.67,"""0""","""A Consultar""","""Julia Abadin """,null,null,null,null,null,"""9412.24""",2143.82,"""RESI""","""NA""","""NA""","""NA""","""NA""","""DISPONIBLE COM…",null,"""LIBRE CON POSE…",null,null,null,null,null,null,null,0.1,null,null,"""La finca tiene…",null,"""VENDER""","""KXA""",null,"""N:\CoralHudson…","""60513529_CH_PU…","""Success""",null,"""6079256""",null,"""60513529""","""TERRENO URBANO…","""AV. PORTUS GAD…",null
1473,"""5473800""",null,300000.0,603568.45,null,"""A consultar""","""Paula Piedrahi…",null,null,null,null,null,"""1396""",1643.54,"""RESI""","""NA""","""NA""","""NA""","""NA""","""DISPONIBLE COM…",null,"""LIBRE P

In [5]:
from conf.settings import DATABASE_FILE
import duckdb

query = """select column_name, data_type, table_name from information_schema.columns where table_name in ('ws_hist_offers', 'ws_current_offers');"""

with duckdb.connect(DATABASE_FILE.as_posix()) as db:
    db.sql(query).df().to_clipboard()

# 🐍 USAR PYTHON-CALAMINE!!

In [1]:
from python_calamine import CalamineWorkbook
import glob
import time
import os
import polars as pl

dos_ficheros = [
    "N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2023/20230210/20230210 OF 5525400 PROMO 60466091 Nave industrial en Arakundiaga.xlsx",
    "N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2023/20230707/20230707_OF_5549328_UR_60460925_SUELO_AV.BENEMIS 29 PILES_VALENCIA.xlsx",
]
ofertas_random = glob.glob(
    "N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2024/**/[!~$]*.xlsx"
)

In [128]:
# los datos a conseguir/igualar son los siguientes
import json

with open("./conf/xy_offer_labels.json", "r") as rjs:
    cnf_raw = rjs.read()
conf = json.loads(cnf_raw)

In [142]:
from typing import List
import re

tipos_datos = {
    "offer_id": pl.Float32,
    "offer_price": pl.Float32,
    "appraisal_price": pl.Float32,
    "sap_price": pl.Float32,
    "web_price": pl.Float32,
    "due_diligence_percent": pl.Float32,
    "contract_deposit_percent": pl.Float32,
    "public_deed_percent": pl.Float32,
    "land_area": pl.Float32,
    "buildable_area": pl.Float32,
    "client_phone": pl.Float32,
}

second_pass_cast = {
    "offer_id": pl.UInt32,
    "client_phone": pl.UInt64,
}


def find_shtname_from_pattern(sheet_list: List[str], sheet_pattern: str) -> str:
    p = re.compile(sheet_pattern, flags=re.IGNORECASE)
    x = list(filter(None, map(p.search, sheet_list)))[0]
    return x.string  # attribute of a re.match object


def get_valid_data(
    sheet_as_values_list: List[str], idx_column_to_count: int
) -> List[str]:
    hdrs = sheet_as_values_list[0]
    valid_headers = list(filter(lambda x: len(x) > 0, hdrs))
    raw_data = list(
        zip(*sheet_as_values_list)
    )[
        : len(valid_headers)
    ]  # <<< limit the headers to a fixed number if the reading fails bcause of formatting
    # raw_data = list(zip(*sheet_as_values_list]))[: len(valid_headers)]
    actual_height = len(
        list(filter(lambda x: x != "", raw_data[idx_column_to_count]))
    )  # <<< dynamic limit to the raw data
    # check the lenght of non-empty items in the first or second column, then crop the data up to that number
    valid_data = list(zip(*sheet_as_values_list[1:actual_height]))[: len(valid_headers)]
    return valid_headers, valid_data


def get_idx_of_pattern_col(pat: str, sheet_as_values_list: List[str]) -> int:
    col_pattern = re.compile(pat, re.IGNORECASE)
    winner = list(
        filter(
            None,
            map(
                col_pattern.search, list(map(lambda x: str(x), sheet_as_values_list[0]))
            ),
        )
    )[0]
    # return sheet_as_values_list.index(winner)
    return sheet_as_values_list[0].index(winner.string)

Aqui hace falta una funcion que arregle los datos antes de volcarlos a DuckDB

In [ ]:
# lo que da problemas por ahora es
# 1) el campo "fecha" (separado por puntos, no por guiones o por barras hacia delante)
# 2) las fechas que no existan, sustituirlas por la extraida de la ruta completa


### 1. Datos de la oferta con Calamine!

In [144]:
data = []
final_data = []

start = time.perf_counter()


for idx, item in enumerate(ofertas_random):
    file_name_short = os.path.basename(item)
    # print(f"Opening file {idx}: {item}")
    print(idx, item)
    wb_inmemory = CalamineWorkbook.from_path(item)
    nombre_hoja_ficha = find_shtname_from_pattern(wb_inmemory.sheet_names, "ficha")
    # Implementar regex y obtener indice
    datitos = wb_inmemory.get_sheet_by_name(nombre_hoja_ficha).to_python()
    inicio = get_idx_of_pattern_col("oferta$", list(zip(*datitos)))
    data = datitos[inicio:]
    erres = {}
    for label, coords in conf.items():
        x, y = coords
        # print(f"{label} is:", wb_prueba[x][y])
        erres[label] = data[x - inicio - 1][y - 1]
    # part 2, the sap data
    try:
        nombre_hoja_sap = find_shtname_from_pattern(wb_inmemory.sheet_names, "sap")
    except IndexError as e:
        print("Error:", e)
        nombre_hoja_sap = find_shtname_from_pattern(wb_inmemory.sheet_names, "oferta")

    hoja = CalamineWorkbook.from_path(item).get_sheet_by_name(nombre_hoja_sap)

    if hoja.total_height > 1:
        rows_to_select = hoja.total_height
    else:
        rows_to_select = None
    data_rows = hoja.to_python(skip_empty_area=False, nrows=rows_to_select)
    idx_of_ur_col = get_idx_of_pattern_col("registral", data_rows)
    _, valid_data = get_valid_data(data_rows, idx_of_ur_col)
    df_data = (
        pl.DataFrame(erres)
        .with_columns(
            [
                pl.when(pl.col(pl.Utf8).str.len_bytes() == 0)
                .then(None)
                .otherwise(pl.col(pl.Utf8))
                .name.keep()
            ]
        )
        # casting, relaxed first then enforced
        .cast(pl.String)
        .cast(tipos_datos, strict=False)
        .cast(second_pass_cast)
    )

    # get the sap data metrics/aggregates
    try:
        df_sap = pl.DataFrame(valid_data, strict=False)
    except pl.ComputeError as e:
        print("Error creating dataframe", e)
        flipped = list(zip(*valid_data))[:5]
        valid_data = list(zip(*flipped))
        print(valid_data)
        df_sap = pl.DataFrame(valid_data, strict=False)
    try:
        df_sap = df_sap.select(
            pl.col(f"column_{idx_of_ur_col}")
            .cast(pl.UInt32)
            .implode()
            .list.unique()
            .alias("urs_unicos"),
            pl.col(f"column_{idx_of_ur_col}")
            .implode()
            .list.unique()
            .list.len()
            .cast(pl.UInt16)
            .alias("total_urs"),
        )
    except pl.ComputeError as e:
        print("Error in file:", item, "for column index", idx_of_ur_col)
        print(df_sap)
        raise e
    except pl.ColumnNotFoundError as e:
        print("Error in file:", item, "for column index", idx_of_ur_col)
        print(df_sap)
        raise e
    united_df = pl.concat([df_data, df_sap], how="horizontal").with_columns(
        pl.col("delegate").str.to_titlecase().name.keep(),
        pl.col("client_name").str.to_titlecase().name.keep(),
        pl.col("svh_recommendation").str.to_titlecase().name.keep(),
        pl.col("client_email").str.to_lowercase().name.keep(),
        full_path=pl.lit(item),
        file_name=pl.lit(file_name_short),
    )

    final_data.append(united_df)

print(
    f"Done. Loaded {len(ofertas_random)} files in {time.perf_counter() - start} seconds"
)

0 N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2024\20230423\20240423 OFERTA 5579533 PROMO 6073945 WIP ANDRATX.xlsx
1 N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2024\20230423\20240423 OFERTA 5584147 PROMO 6073609 VIVIENDA, ROTA, CADIZ.xlsx
2 N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2024\20230423\20240423_OF_5583510_PROMO 6082032_SUELO_CL. SUP-8 RUEDOS DE CONSOLACION M23-24_UTRERA_SEVILLA.xlsx
3 N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2024\20230423\20240423_OF_5583836_UR_60531816_ SUNC_CL_SAN PEDRO_LLERENA_CH.xlsx
4 N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2024\20240102\20231219_OF 5570280 UR 60529132_LOCAL_CL.REPUBLICA 3_SABADELL.xlsx
5 N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2024\20240102\20231227_OF 5570700_ UR 60456464_PARCELA 12.1 STR SUB-CC-9.2 LAS PALOMAS (BADAJOZ) CH.xlsx
6 N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2024\20240

In [155]:
from conf.functions import retrieve_all_info

# comprobemos ahora las posiciones relativas y mejoremos la ubicacion
# ficherito = "N:/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2024/20240424/20240424_OF_ 55583845_Promo_60473086_Luis Vives_SCTF.xlsx"
ficherito = "//EURFL01/advisors.hal/non-hudson/Coral Homes/CoralHudson/1. AM/8. Wholesale Channel/Ofertas recibidas SVH/2024/20240429/OF_5583734_PR_6077421_PROMO SUELO_URB FUENTE ELVIRA_GALAPAGAR_MADRID.xlsx"
datitos = CalamineWorkbook.from_path(ficherito).get_sheet_by_index(0).to_python()
inicio = get_idx_of_pattern_col("oferta$", list(zip(*datitos)))
# get_valid_data(datitos, inicio)
# retrieve_all_info(
#     [ficherito], config_file="./conf/xy_offer_labels.json", fallback_sample_limit=5
# )
datos = datitos[inicio:]
for idx, val in enumerate(datos):
    print(idx, val)


0 ['Num. \r\nOFERTA', 'Precio web', 'TASACIÓN\r\nV.MERCADO', 'OFERTA', '', 'DOSSIER COMERCIAL', 'PRECIO SAP', 'OFERTANTE', '', '', '', '', '']
1 [5583734.0, 'A/C', 391228.9999999999, 224000.0, '', 'SI', 0.0, 'NOMBRE', '', 'CESCASA INMOB Y GESTIÓN DE VIVIENDAS S.L', '', '', '']
2 ['', '', '', '', '', '', '', 'CIF', '', 'B82212812', '', '', '']
3 ['Delegado', 'TOMAS LOPEZ HERRERA', '', '', '', 'Publicado web SvH/No publicado en web', 'ROLL UP', 'TELEFONO', '', 918425582.0, '', '', '']
4 ['Segmentación', 'B', '', '', '', 'Si', '', 'EMAIL', '', '\t\r\nCESCASA@CESCASA.COM', '', '', '']
5 ['Reunion Comprador', 'NO', '', '', '', '', '', 'DIRECCION', '', 'C/ Galapagar 4 _  28270 Colmenarejo', '', '', '']
6 ['DESCRIPCIÓN DEL ACTIVO', '', '', '', '', '', '', 'CONDICIONES OFERTANTE', '', '', '', '', '']
7 ['Conjunto de suelos rústicos destinados a uso residencial en Urbanización Fuente Elvira, Galapagar.\r\n\r\nSuelos rústicos situados al suroeste de Galapagar, lindando con el término municipal d

In [ ]:
# usado para traer los datos de sap correctamente!!
def get_valid_data(
    sheet_as_values_list: List[str], idx_column_to_count: int
) -> List[str]:
    hdrs = sheet_as_values_list[0]
    valid_headers = list(filter(lambda x: len(x) > 0, hdrs))
    raw_data = list(zip(*sheet_as_values_list))[: len(valid_headers)]
    # raw_data = list(zip(*sheet_as_values_list]))[: len(valid_headers)]
    actual_height = len(list(filter(lambda x: x != "", raw_data[idx_column_to_count])))
    # check the lenght of non-empty items in the first or second column, then crop the data up to that number
    valid_data = list(zip(*sheet_as_values_list[1:actual_height]))[: len(valid_headers)]
    return valid_headers, valid_data


errorcito = ofertas_random[24]
print(errorcito)
errorcito = CalamineWorkbook.from_path(errorcito).get_sheet_by_index(1).to_python()
idx_of_ur_col = get_idx_of_pattern_col("registral", data_rows)
get_valid_data(errorcito, idx_of_ur_col)


In [145]:
for i, d in enumerate(final_data):
    print(i, "\t", len(d.columns), "\t", d.dtypes)

0 	 39 	 [UInt32, Float32, Float32, Float32, Float32, String, String, String, String, String, String, Float32, Float32, Float32, String, String, String, String, String, String, String, String, String, Float32, Float32, String, String, String, String, String, String, String, UInt64, String, String, List(UInt32), UInt16, String, String]
1 	 39 	 [UInt32, Float32, Float32, Float32, Float32, String, String, String, String, String, String, Float32, Float32, Float32, String, String, String, String, String, String, String, String, String, Float32, Float32, String, String, String, String, String, String, String, UInt64, String, String, List(UInt32), UInt16, String, String]
2 	 39 	 [UInt32, Float32, Float32, Float32, Float32, String, String, String, String, String, String, Float32, Float32, Float32, String, String, String, String, String, String, String, String, String, Float32, Float32, String, String, String, String, String, String, String, UInt64, String, String, List(UInt32), UInt16, Strin

In [146]:
final_data[7]

offer_id,offer_price,appraisal_price,sap_price,web_price,delegate,segment,buyer_meeting,listed,rollup,dossier,due_diligence_percent,contract_deposit_percent,public_deed_percent,conditions,offer_description,svh_opinion,svh_recommendation,lead_type,lead_id,commercial_status,possession_status,legal_status,land_area,buildable_area,main_use,planning_works,reparcelation_project,urbanization_project,urban_works,client_name,client_tax_id,client_phone,client_email,client_address,urs_unicos,total_urs,full_path,file_name
u32,f32,f32,f32,f32,str,str,str,str,str,str,f32,f32,f32,str,str,str,str,str,str,str,str,str,f32,f32,str,str,str,str,str,str,str,u64,str,str,list[u32],u16,str,str
5570711,680000.0,1.2796e6,null,null,"""Alejandro Bosch""","""B""","""40%FFPP/60% FINANCIACIÓN 45 di…","""SI""","""No""","""No""",null,null,null,null,"""La PC del asunto es un edifici…","""Segunda oferta recibida por es…","""Contraofertar""","""API""","""A070013004""","""DISPONIBLE COMERCIALIZAR""","""LIBRE CON POSESIÓN""","""TIT. INSCRITO""",null,1324.26001,"""RESI""","""NA""","""NA""","""NA""","""NA""","""Pablo Javier Lopez Hernandez""","""78517643 J""",696623168,"""pablojaloro@hotmail.com""","""Rambla Roger de Lauria, 75 351…","[60451523, 60451537, … 60492078]",37,"""N:/CoralHudson/1. AM/8. Wholes…","""20231227_OF_5570711_PC_6073207…"


In [147]:
unidos = pl.concat(final_data)
unidos

offer_id,offer_price,appraisal_price,sap_price,web_price,delegate,segment,buyer_meeting,listed,rollup,dossier,due_diligence_percent,contract_deposit_percent,public_deed_percent,conditions,offer_description,svh_opinion,svh_recommendation,lead_type,lead_id,commercial_status,possession_status,legal_status,land_area,buildable_area,main_use,planning_works,reparcelation_project,urbanization_project,urban_works,client_name,client_tax_id,client_phone,client_email,client_address,urs_unicos,total_urs,full_path,file_name
u32,f32,f32,f32,f32,str,str,str,str,str,str,f32,f32,f32,str,str,str,str,str,str,str,str,str,f32,f32,str,str,str,str,str,str,str,u64,str,str,list[u32],u16,str,str
5579533,270000.0,272217.15625,450000.0,450000.0,"""Jose Maria Abril""",null,"""SI ""","""SI""","""NO""","""NO""",null,0.0,1.0,"""E.P. 30 DIAS TRAS ACEPTACIÓN D…","""Obra en curso que se constituy…","""Oferta procedente del departam…","""Vender""","""SERVIHABITAT""",null,"""DISPONIBLE COMERCIALIZAR""","""LIBRE CON POSESIÓN""","""TIT. INSCRITO""",null,1345.459961,"""RESI""","""NA""","""NA""","""NA""","""NA""","""Ison Terrasa Inversions 2006 S…","""B57417362""",971214016,"""canmetles@gmail.com""","""C/ Jaume III 20, Palma Mallorc…","[60462639, 60462650, … 60478228]",8,"""N:/CoralHudson/1. AM/8. Wholes…","""20240423 OFERTA 5579533 PROMO …"
5584147,135000.0,124870.0,0.0,0.0,"""Jose Maria Abril""",null,"""SI ""","""SI""","""NO""","""NO""",null,0.0,1.0,"""E.P. 30 DIAS TRAS ACEPTACIÓN D…","""Vivienda + garaje y trastero e…","""Oferta procedente del departam…","""Vender""","""SERVIHABITAT""",null,"""DISPONIBLE COMERCIALIZAR""","""DENEGADA POSESION JUZGADO""","""TIT. INSCRITO""",null,103.0,"""RESI""","""NA""","""NA""","""NA""","""NA""","""Telcosur Almudeyne Sl""","""B90354671""",677207104,"""cotashome@gmail.com""","""C/ Arquitectura 2 Sevilla""","[60460896, 60460908, 60460931]",3,"""N:/CoralHudson/1. AM/8. Wholes…","""20240423 OFERTA 5584147 PROMO …"
5583510,485000.0,2563422.5,2.8498e6,2.8498e6,"""Fernando Ruiz-cabello""","""D""","""SI""",null,"""SI""","""SI""",null,0.0,1.0,"""E.P. 100% A LOS 40 DIAS DESDE …","""SUP 8. MANZANAS M23 y M24 SUEL…","""Se trata de un suelo finalista…","""Vender""","""SERVIHABITAT""",null,"""DISPONIBLE COMERCIALIZAR""","""LIBRE CON POSESIÓN""","""TIT. INSCRITO""",null,15000.0,"""RESI""","""NA""","""NA""","""NA""","""NA""","""S. J. De Inversiones, S.l. .""","""B-41724402""",610715328,"""jmsanchez@ibernegociosinmobili…","""CARNICEROS 32, EL VISO DEL ALC…","[60520231, 60520252, … 60533772]",185,"""N:/CoralHudson/1. AM/8. Wholes…","""20240423_OF_5583510_PROMO 6082…"
5583836,95000.0,484676.78125,390000.0,390000.0,"""Antonio Expósito Haza""","""D""",null,null,"""SI""","""SI""",null,null,1.0,"""E.P. antes 30/05/2024""","""Parcela de SUNC dentro de 3 Un…","""Dados los precios de venta de …","""Vender""","""API""","""A001025503""","""DISPONIBLE COMERCIALIZAR""","""LIBRE POSESIÓN ACTA""","""TIT. INSCRITO""",13149.0,8109.0,"""RESI""","""NA""","""NA""","""NA""","""NA""","""Atoga Extrem Sl""","""B56609480 """,607594432,"""javier@javiergaspar.com""","""PARQUE DE DOÑANA 15, BADAJOZ""",[60531816],1,"""N:/CoralHudson/1. AM/8. Wholes…","""20240423_OF_5583836_UR_6053181…"
5570280,305000.0,536000.0,350000.0,null,"""Jordi Sala Arumi""","""A""","""NO""",null,"""NO""","""SI""",null,0.1,0.9,"""E.P. 22/01/2024""","""Local en edificio terminado de…","""Recomendamos esta oferta por t…","""Vender""","""API""","""A070013599""","""DISPONIBLE COMERCIALIZAR""","""REVISADA ENTREGA VOLUNTARIA DE…","""TIT. INSCRITO""",null,214.399994,"""COMERCIAL""","""NA""","""NA""","""NA""","""NA""","""Sicosa Home Slu""","""B44578110""",610523968,"""gerencia@clinicasanident.com""","""SANT QUIRZE DEL VALLES""",[60529132],1,"""N:/CoralHudson/1. AM/8. Wholes…","""20231219_OF 5570280 UR 6052913…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
5578718,250000.0,443305.5,0.0,null,"""Antonio Expósito Haza""","""D""","""Si""",null,"""SI""","""SI""",null,0.0,1.0,"""EP MAX 

| cols1           | cols2                     | cols3                       |
|-----------------|---------------------------|-----------------------------|
| offer_id        | main_use                  | client_email                |
| offer_date      | planning_works            | client_address              |
| offer_price     | land_readjustment_project | due_diligence               |
| appraisal_price | urbanization_project      | contract_deposit            |
| sap_price       | urban_works               | public_deed                 |
| web_price       | commercial_status         | conditions_details          |
| delegate        | legal_status              | offer_description_long      |
| segment         | possession_status         | servihabitat_opinion        |
| buyer_meeting   | client_name               | servihabitat_recommendation |
| listed_y_n      | client_description        | offer_lead_type             |
| rollup_y_n      | client_tax_id             | offer_lead_id               |
| dossier         | client_phone              |                             |
| land_area       |                           |                             |
| buildable_area  |                           |                             |

#### 1.1 Integrar los resultados en duckdb

In [148]:
import duckdb

duckdb.sql("""select * from unidos""").show()

┌──────────┬─────────────┬─────────────────┬───┬───────────┬──────────────────────┬──────────────────────┐
│ offer_id │ offer_price │ appraisal_price │ … │ total_urs │      full_path       │      file_name       │
│  uint32  │    float    │      float      │   │  uint16   │       varchar        │       varchar        │
├──────────┼─────────────┼─────────────────┼───┼───────────┼──────────────────────┼──────────────────────┤
│  5579533 │    270000.0 │       272217.16 │ … │         8 │ N:/CoralHudson/1. …  │ 20240423 OFERTA 55…  │
│  5584147 │    135000.0 │        124870.0 │ … │         3 │ N:/CoralHudson/1. …  │ 20240423 OFERTA 55…  │
│  5583510 │    485000.0 │       2563422.5 │ … │       185 │ N:/CoralHudson/1. …  │ 20240423_OF_558351…  │
│  5583836 │     95000.0 │       484676.78 │ … │         1 │ N:/CoralHudson/1. …  │ 20240423_OF_558383…  │
│  5570280 │    305000.0 │        536000.0 │ … │         1 │ N:/CoralHudson/1. …  │ 20231219_OF 557028…  │
│  5570700 │    594400.0 │       4341

### 2. Calamine -> Polars -> DuckDB

# Plotting funciones hipotecarias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Variables
M = 321.64  # Monthly payment
n = 30 * 12  # Loan term in months

# Range of annual interest rates
r_annual = np.linspace(0.01, 0.1, 100)  # From 1% to 10%

# Convert annual interest rate to monthly
r = r_annual / 12

# Calculate the purchase price
P = M * ((1 + r) ** n - 1) / (r * (1 + r) ** n)

# Plot
plt.plot(r_annual, P)
plt.xlabel("Annual Interest Rate")
plt.ylabel("Purchase Price")
plt.title("Purchase Price vs Interest Rate")
plt.grid(True)
plt.show()

In [ ]:
# Variables
P = 100000  # Purchase price
n = 30 * 12  # Loan term in months

# Range of annual interest rates
r_annual = np.linspace(0.01, 0.1, 100)  # From 1% to 10%

# Convert annual interest rate to monthly
r = r_annual / 12

# Calculate the monthly payment
M = P * r * (1 + r) ** n / ((1 + r) ** n - 1)

# Plot
plt.plot(r_annual, M)
plt.xlabel("Annual Interest Rate")
plt.ylabel("Monthly Payment")
plt.title("Monthly Payment vs Interest Rate")
plt.grid(True)
plt.show()

In [ ]:
# Variables
M = 1000  # Monthly payment
n = 30 * 12  # Loan term in months

# Range of annual interest rates
r_annual = np.linspace(0.01, 0.05, 100)  # From 1% to 10%

# Convert annual interest rate to monthly
r = r_annual / 12

# Calculate the purchase price
P = M * ((1 + r) ** n - 1) / (r * (1 + r) ** n)

# Calculate the derivative of the purchase price with respect to the interest rate
dP_dr = np.gradient(P, r)

# Plot purchase price
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.plot(r_annual, P, label="Purchase Price")
plt.xlabel("Annual Interest Rate")
plt.ylabel("Purchase Price")
plt.title("Purchase Price and its Derivative vs Interest Rate")
plt.grid(True)

# Plot derivative
plt.subplot(2, 1, 2)
plt.plot(r_annual, dP_dr, label="dP/dr", color="r")
plt.xlabel("Annual Interest Rate")
plt.ylabel("dP/dr")
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Variables
M = 321.64  # Monthly payment
n = 30 * 12  # Loan term in months

# Range of annual interest rates
r_annual = np.linspace(0.01, 0.1, 100)  # From 1% to 10%

# Convert annual interest rate to monthly
r = r_annual / 12

# Calculate the purchase price
P = M * ((1 + r) ** n - 1) / (r * (1 + r) ** n)

# Calculate the percentage decrease in P
P_diff = np.diff(P) / P[:-1] * 100  # in percentage

# Calculate the cumulative sum of the percentage decreases
P_diff_cumsum = np.cumsum(
    np.abs(P_diff)
)  # absolute value to keep all decreases positive

# Plot Purchase Price
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.plot(r_annual, P, label="Purchase Price")
plt.xlabel("Annual Interest Rate")
plt.ylabel("Purchase Price")
plt.title("Purchase Price and Cumulative % Decreases vs Interest Rate")
plt.grid(True)

# Plot Cumulative % Decreases in P
plt.subplot(2, 1, 2)
plt.plot(r_annual[1:], P_diff_cumsum, label="Cumulative % Decrease", color="r")

# Adding markers for x-axis crossing points
x_label_indices = np.arange(
    0, len(r_annual), len(r_annual) // 10
)  # Indices for x-axis labels
plt.plot(
    r_annual[x_label_indices], P_diff_cumsum[x_label_indices], "bo"
)  # Plot blue circle markers

plt.xlabel("Annual Interest Rate")
plt.ylabel("Cumulative % Decrease")
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Variables
M = 932.11  # Monthly payment
n = 30 * 12  # Loan term in months

# Range of annual interest rates
r_annual = np.linspace(0.01, 0.1, 10)  # From 1% to 10%, only 10 points for clarity

# Convert annual interest rate to monthly
r = r_annual / 12

# Calculate the purchase price
P = M * ((1 + r) ** n - 1) / (r * (1 + r) ** n)

# Calculate the percentage decrease in P with respect to initial P
P_initial = P[0]
P_diff = (P - P_initial) / P_initial * 100  # in percentage

# Plot Purchase Price
fig, ax1 = plt.subplots(figsize=(10, 6))

color = "tab:blue"
ax1.set_xlabel("Annual Interest Rate")
ax1.set_ylabel("Purchase Price", color=color)
ax1.plot(r_annual, P, label="Purchase Price", color=color)
ax1.tick_params(axis="y", labelcolor=color)
plt.grid(True)

# Create a second y-axis that shares the same x-axis
ax2 = ax1.twinx()

color = "tab:red"
ax2.set_ylabel("% Decrease", color=color)
ax2.plot(r_annual, P_diff, label="% Decrease", color=color)
ax2.tick_params(axis="y", labelcolor=color)

# Annotate data points
for x, y1, y2 in zip(r_annual, P, P_diff):
    ax1.annotate(
        f"{y1:.0f}",
        (x, y1),
        textcoords="offset points",
        xytext=(0, 15),
        ha="center",
        size=8,
    )
    ax2.annotate(
        f"{y2:.0f}%",
        (x, y2),
        textcoords="offset points",
        xytext=(0, -15),
        ha="center",
        size=8,
        color=color,
    )

plt.title("Purchase Price and % Decreases vs Interest Rate")
plt.grid(True)
plt.tight_layout()
plt.show()